In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import numpy as np
import pandas as pd
import ast
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [27]:
file_path1 = "/content/drive/MyDrive/Movie_Recommendation_Systems/Movie_Data/tmdb_5000_movies.csv"
file_path2 = "/content/drive/MyDrive/Movie_Recommendation_Systems/Movie_Data/tmdb_5000_credits.csv"
movies_df = pd.read_csv(file_path1)
credits_df = pd.read_csv(file_path2)

In [28]:
def convert(obj):
    return [i['name'] for i in ast.literal_eval(obj)]

def convert2(obj):
    if isinstance(obj, str):
        obj = ast.literal_eval(obj)
    return [i for i in obj[:3]]

def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

def stem(text):
    ps = PorterStemmer()
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:11]
    for i in movies_list:
      print(new_df.iloc[i[0]].title)

In [29]:
def main(movies_df, credits_df):
    movies = movies_df.merge(credits_df,on='title')
    movies = movies[['movie_id', 'title','overview','genres','keywords','cast','crew']]
    movies.dropna(inplace=True)
    movies['genres'] = movies['genres'].apply(lambda x: convert(x))
    movies['keywords'] = movies['keywords'].apply(lambda x : convert(x))
    movies['cast'] = movies['cast'].apply(lambda x : convert2(x))
    movies['crew'] = movies['crew'].apply(fetch_director)
    movies['overview'] = movies['overview'].apply(lambda x: x.split())
    movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
    movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
    movies['cast'] = movies['cast'].apply(lambda x: [i['name'].replace(" ", "") for i in x])
    movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x])
    movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
    new_df = movies[['movie_id','title','tags']]
    new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))
    new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())
    new_df['tags'] = new_df['tags'].apply(stem)
    cv = CountVectorizer(max_features=5000, stop_words='english')
    vectors = cv.fit_transform(new_df['tags']).toarray()
    similarity = cosine_similarity(vectors)
    movie_name = input("Enter the name of the movie for recommendation: ")
    reco_output = recommend(movie_name)
    return reco_output

In [30]:
if __name__ == "__main__":
    recommended_names = main(movies_df, credits_df)
    print(recommended_names)

Enter the name of the movie for recommendation: Avatar
Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.
Battle: Los Angeles
Predators
Small Soldiers
Jupiter Ascending
Meet Dave
None


In [31]:
import pickle
pickle.dump(new_df, open('movies.pkl', 'wb'))